In [1]:
#from hiive.mdptoolbox import mdp, example
from mdp import MDP

In [2]:
import gym, gym_walk, numpy as np
env = gym.make('WalkFive-v0')
pi = lambda x: np.random.randint(2)

In [3]:
env.observation_space.n

7

In [4]:
env.reset()

/opt/homebrew/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:195: UserWarning: WARN: The result returned by `env.reset()` was not a tuple of the form `(obs, info)`, where `obs` is a observation and `info` is a dictionary containing additional information. Actual type: `<class 'int'>`
  logger.warn(


3

In [5]:

def td(pi, env, gamma=1.0, alpha=0.01, n_episodes=100000):
    V = np.zeros(env.observation_space.n)
    for t in range(n_episodes):
        state = env.reset()
        done = False
        while not done:
            action = pi(state)
            # s_prime, reward, done, info = self.env.step(action)
            next_state, reward, done, info, _ = env.step(action)
            td_target = reward + gamma * V[next_state] * (not done)
            td_error = td_target - V[state]
            V[state] = V[state] + alpha * td_error
            state = next_state
    return V

V = td(pi, env)
V

/opt/homebrew/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:219: DeprecationWarning: WARN: Core environment is written in old step API which returns one bool instead of two. It is recommended to rewrite the environment with new step API. 
  logger.deprecation(


ValueError: not enough values to unpack (expected 5, got 4)

In [6]:
class QLearningAgent(object):
    def __init__(self):
        self.Q = None
        self.alpha = 0.2  # param
        self.gamma = 0.90
        self.epsilon = 0.25 # param
        self.n_episodes = 1_000 # param
        self.env = gym.make("WalkFive-v0").env

    def solve(self):
        """Create the Q table"""
        self.Q = np.zeros([self.env.observation_space.n, self.env.action_space.n])
        for n in range(self.n_episodes):
            # Initialize S
            s = self.env.reset()
            done = False
            
            # Loop for each step of episode
            while not done:
                # Choose A from S
                if np.random.rand() <= self.epsilon:
                    a = np.random.randint(self.env.action_space.n)
                else:
                    a = np.argmax(self.Q[s, :])
                
                # Take action A
                s_prime, reward, done, info = self.env.step(a)
                
                # Update Q Table
                self.Q[s, a] += self.alpha * (reward + self.gamma * self.Q[s_prime, np.argmax(self.Q[s_prime, :])] - self.Q[s, a])
                s = s_prime
        return reward

    def Q_table(self, state, action):
        """return the optimal value for State-Action pair in the Q Table"""
        return self.Q[state][action]

In [7]:
agent = QLearningAgent()
agent.solve()

1.0

In [8]:
agent.Q

array([[0.          , 0.          ],
       [0.          , 0.1292895743],
       [0.          , 0.7254857673],
       [0.6376480195, 0.81        ],
       [0.7207472225, 0.9         ],
       [0.8082280821, 1.          ],
       [0.          , 0.          ]])

In [12]:
walk = MDP(environment='WalkFive-v0', convergence_threshold=0.00001, grid=False)
print('Random Walk')

walk.value_iteration(iterations_to_save=[1, 2, 4, 6, 8, 10, 12], visualize=False)
walk.policy_iteration(iterations_to_save=[1, 2, 4, 8, 10], visualize=False)
# walk.Q_learning(num_episodes=1000, learning_rate_decay=0.995, epsilon_decay=0.995, visualize=True)

Random Walk

Value-iteration

---> converged at iteration 6 in 0.0008 seconds

---> optimal V = 
[0.     0.6561 0.729  0.81   0.9    1.     0.    ]

---> optimal policy = 
[[1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]]

---> optimal policy actions = 
[0 1 1 1 1 1 0]

Policy-iteration
PI iteration 1
PI iteration 2

---> converged at iteration 2 in 0.0016 seconds

---> optimal V = 
[0.     0.6561 0.729  0.81   0.9    1.     0.    ]

---> optimal policy = 
[[1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]]

---> optimal policy actions = 
[0 1 1 1 1 1 0]


In [10]:
taxi = MDP(environment='Taxi-v3', convergence_threshold=0.0001, grid=True)
print('Taxi')

taxi.value_iteration(iterations_to_save=[1, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50], visualize=True)
taxi.policy_iteration(iterations_to_save=[0, 1, 8, 16, 32, 64, 128], visualize=True)
taxi.Q_learning(num_episodes=10000, learning_rate_decay=0.995, epsilon_decay=0.995, visualize=True)

Taxi

Value-iteration

---> converged at iteration 63 in 0.8382 seconds

---> optimal V = 
[ 89.4734925232  32.8198671487  55.2643920487  37.5775540189
   8.4324828228  32.8198671487   8.4323823484  15.2844751006
  32.8199676232  18.0940143619  55.2643920487  21.2154953668
  12.7561965968  18.0940143619  12.7560961224  37.5775540189
 100.5261432709  37.5776918439  62.5160528439  42.864033517
  79.5261432709  28.5378804339  48.7379528439  32.819798617
  10.4805769371  37.5776918439  10.4804865101  18.0939458301
  28.5379708609  15.2846129257  48.7379528439  18.0939458301
  15.2847033527  21.2156331918  15.2846129257  42.864033517
  89.4735289438  42.8641575595  55.2644475595  48.7378911653
  42.8642389438  12.7561516331  24.6840923905  15.2845512471
  24.6841737748  70.5734475595  24.6840923905  37.5776301653
  24.6841737748  12.7561516331  42.8641575595  15.2845512471
  18.0941512569  24.6840923905  18.0940698726  48.7378911653
  48.7380760494  79.5261028035  48.7380028035  55.26439204

AttributeError: 'TaxiEnv' object has no attribute 'nrow'

<Figure size 1000x1000 with 0 Axes>

In [13]:
env

<TimeLimit<OrderEnforcing<PassiveEnvChecker<WalkEnv<WalkFive-v0>>>>>